In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

In [6]:

df_train = pd.read_csv("./train/train_2024.csv", quoting=3, index_col=0)
df_val = pd.read_csv("./dev/dev_2024.csv", quoting=3, index_col=0) 

y_train = df_train['label'].values
X_train = df_train['text'].values

y_val = df_val['label'].values
X_val = df_val['text'].values

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

# Convert class weights to a dictionary
weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

#val_labels = df_val['label'].values
#val_text = df_val['text'].values

# Compute TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_train_tfidf = vectorizer.transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Initialize classifiers
logreg = LogisticRegression(class_weight=weight_dict, max_iter=200000 )
knn = KNeighborsClassifier(n_neighbors=2)  # Does not support class_weight
decision_tree = DecisionTreeClassifier()  # Supports via min_weight_fraction_leaf but not directly class_weight
svm = SVC(class_weight=weight_dict)
random_forest = RandomForestClassifier(n_estimators=100, class_weight=weight_dict)

models = [logreg, knn, decision_tree, svm, random_forest]
model_names = ['Logistic Regression', 'K-Nearest Neighbors', 'Decision Tree', 'SVM', 'Random Forest']

# Train and evaluate each classifier
for model, name in zip(models, model_names):
    model.fit(X_train_tfidf, y_train)  # Train model
    y_pred = model.predict(X_val_tfidf)  # Predict on test set
    print(f"{name} Classification Report:")
    print(classification_report(y_val, y_pred))

df_train = pd.read_csv("./train/train_2024.csv", quoting=3, index_col=0)
df_val = pd.read_csv("./dev/dev_2024.csv", quoting=3, index_col=0) 

y_train = df_train['label'].values
X_train = df_train['text'].values

y_val = df_val['label'].values
X_val = df_val['text'].values



Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      7019
           1       0.89      0.89      0.89      3981

    accuracy                           0.92     11000
   macro avg       0.91      0.91      0.91     11000
weighted avg       0.92      0.92      0.92     11000

